<a href="https://colab.research.google.com/github/meka-soluciones/Inteligencia-Artificial/blob/main/Code%20Logica%20Fuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 9.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown
import warnings
warnings.filterwarnings("ignore")

# VARIABLES DIFUSAS
temperatura = ctrl.Antecedent(np.arange(0, 41, 1), 'temperatura')
humedad = ctrl.Antecedent(np.arange(0, 101, 1), 'humedad')
riego = ctrl.Consequent(np.arange(0, 101, 1), 'riego')

# FUNCIONES DE MEMBRESÍA
temperatura['baja'] = fuzz.trapmf(temperatura.universe, [0, 0, 10, 20])
temperatura['media'] = fuzz.trimf(temperatura.universe, [15, 25, 35])
temperatura['alta'] = fuzz.trapmf(temperatura.universe, [30, 35, 40, 40])

humedad['seca'] = fuzz.trapmf(humedad.universe, [0, 0, 30, 50])
humedad['media'] = fuzz.trimf(humedad.universe, [30, 50, 70])
humedad['humeda'] = fuzz.trapmf(humedad.universe, [60, 80, 100, 100])

riego['bajo'] = fuzz.trapmf(riego.universe, [0, 0, 20, 40])
riego['medio'] = fuzz.trimf(riego.universe, [30, 50, 70])
riego['alto'] = fuzz.trapmf(riego.universe, [60, 80, 100, 100])

# REGLAS
regla1 = ctrl.Rule(temperatura['alta'] & humedad['seca'], riego['alto'])
regla2 = ctrl.Rule(temperatura['media'] & humedad['media'], riego['medio'])
regla3 = ctrl.Rule(temperatura['baja'] | humedad['humeda'], riego['bajo'])

sistema_riego = ctrl.ControlSystem([regla1, regla2, regla3])

# DASHBOARD FUNCTION
def fuzzy_dashboard(temp, hum, metodo):
    simulador = ctrl.ControlSystemSimulation(sistema_riego)
    simulador.input['temperatura'] = temp
    simulador.input['humedad'] = hum
    simulador.compute()

    # Cálculo de pertenencias
    x_riego = riego.universe
    act_temp_baja = fuzz.interp_membership(temperatura.universe, temperatura['baja'].mf, temp)
    act_temp_media = fuzz.interp_membership(temperatura.universe, temperatura['media'].mf, temp)
    act_temp_alta = fuzz.interp_membership(temperatura.universe, temperatura['alta'].mf, temp)
    act_hum_seca = fuzz.interp_membership(humedad.universe, humedad['seca'].mf, hum)
    act_hum_media = fuzz.interp_membership(humedad.universe, humedad['media'].mf, hum)
    act_hum_humeda = fuzz.interp_membership(humedad.universe, humedad['humeda'].mf, hum)

    act_regla1 = np.fmin(act_temp_alta, act_hum_seca)
    act_regla2 = np.fmin(act_temp_media, act_hum_media)
    act_regla3 = np.fmax(act_temp_baja, act_hum_humeda)

    salida_regla1 = np.fmin(act_regla1, riego['alto'].mf)
    salida_regla2 = np.fmin(act_regla2, riego['medio'].mf)
    salida_regla3 = np.fmin(act_regla3, riego['bajo'].mf)

    salida_agregada = np.fmax(salida_regla1, np.fmax(salida_regla2, salida_regla3))

    # DEFUZZIFICACIÓN SEGURA
    if np.sum(salida_agregada) == 0:
        resultado = None
        print("⚠️ Ninguna regla se activó. No se puede defuzzificar.")
    else:
        resultado = fuzz.defuzz(x_riego, salida_agregada, metodo)
        print(f"✅ Temperatura: {temp} °C | Humedad: {hum} %")
        print(f"✅ Nivel de riego recomendado ({metodo}): {resultado:.2f} %")

    # GRÁFICA
    plt.figure(figsize=(10, 6))
    plt.plot(x_riego, riego['bajo'].mf, 'b--', label='Riego bajo')
    plt.plot(x_riego, riego['medio'].mf, 'g--', label='Riego medio')
    plt.plot(x_riego, riego['alto'].mf, 'r--', label='Riego alto')
    plt.fill_between(x_riego, salida_regla1, color='red', alpha=0.4, label='Regla 1')
    plt.fill_between(x_riego, salida_regla2, color='green', alpha=0.4, label='Regla 2')
    plt.fill_between(x_riego, salida_regla3, color='blue', alpha=0.4, label='Regla 3')
    plt.plot(x_riego, salida_agregada, 'k', linewidth=2, label='Agregado')

    if resultado is not None:
        plt.axvline(resultado, color='purple', linestyle='--', label=f'Salida: {resultado:.2f} %')

    plt.title(f'Reglas activadas y salida ({metodo})')
    plt.xlabel('Nivel de Riego (%)')
    plt.ylabel('Grado de pertenencia')
    plt.legend()
    plt.grid(True)
    plt.show()

# INTERACTOR
interact(
    fuzzy_dashboard,
    temp=FloatSlider(min=0, max=40, step=1, value=25, description='Temperatura (°C)'),
    hum=FloatSlider(min=0, max=100, step=1, value=50, description='Humedad (%)'),
    metodo=Dropdown(
    options=[
        ('Centroide', 'centroid'),
        ('Media de los máximos', 'mom'),
        ('Minimo del máximo', 'som'),
        ('Maximo del máximo', 'lom')
    ],
    value='centroid',
    description='Método de defuzzificación'
)

)


interactive(children=(FloatSlider(value=25.0, description='Temperatura (°C)', max=40.0, step=1.0), FloatSlider…

<function __main__.fuzzy_dashboard(temp, hum, metodo)>

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown
import warnings
warnings.filterwarnings("ignore")

# VARIABLES DIFUSAS
distancia = ctrl.Antecedent(np.arange(0, 101, 1), 'distancia')
luz_freno = ctrl.Antecedent(np.arange(0, 2, 1), 'luz_freno')
velocidad = ctrl.Antecedent(np.arange(0, 151, 1), 'velocidad')
frenado = ctrl.Consequent(np.arange(0, 101, 1), 'frenado')

# FUNCIONES DE MEMBRESÍA
distancia['corta'] = fuzz.trapmf(distancia.universe, [0, 0, 10, 30])
distancia['media'] = fuzz.trapmf(distancia.universe, [20, 40, 60, 80])
distancia['larga'] = fuzz.trapmf(distancia.universe, [70, 90, 100, 100])

luz_freno['apagada'] = fuzz.trimf(luz_freno.universe, [0, 0, 1])
luz_freno['encendida'] = fuzz.trimf(luz_freno.universe, [0, 1, 1])

velocidad['baja'] = fuzz.gaussmf(velocidad.universe, 30, 10)
velocidad['media'] = fuzz.gaussmf(velocidad.universe, 75, 15)
velocidad['alta'] = fuzz.gaussmf(velocidad.universe, 120, 10)

frenado['nulo'] = fuzz.trimf(frenado.universe, [0, 0, 30])
frenado['moderado'] = fuzz.trimf(frenado.universe, [20, 50, 80])
frenado['fuerte'] = fuzz.trimf(frenado.universe, [70, 100, 100])

# REGLAS
reglas = [
    ctrl.Rule(distancia['corta'] & luz_freno['encendida'] & velocidad['alta'], frenado['fuerte']),
    ctrl.Rule(distancia['corta'] & velocidad['alta'], frenado['fuerte']),
    ctrl.Rule(distancia['media'] & luz_freno['encendida'], frenado['moderado']),
    ctrl.Rule(distancia['larga'] & velocidad['baja'], frenado['nulo']),
    ctrl.Rule(luz_freno['apagada'] & velocidad['baja'], frenado['nulo']),
    ctrl.Rule(distancia['media'] & velocidad['alta'], frenado['moderado']),
    ctrl.Rule(distancia['corta'] & luz_freno['apagada'], frenado['moderado']),
]

sistema_frenado = ctrl.ControlSystem(reglas)

# FUNCIÓN VISUAL INTERACTIVA
def fuzzy_frenado(dist, luz, vel, metodo):
    simulador = ctrl.ControlSystemSimulation(sistema_frenado)
    simulador.input['distancia'] = dist
    simulador.input['luz_freno'] = luz
    simulador.input['velocidad'] = vel
    simulador.compute()

    x_frenado = frenado.universe

    act_dist_corta = fuzz.interp_membership(distancia.universe, distancia['corta'].mf, dist)
    act_dist_media = fuzz.interp_membership(distancia.universe, distancia['media'].mf, dist)
    act_dist_larga = fuzz.interp_membership(distancia.universe, distancia['larga'].mf, dist)
    act_vel_baja = fuzz.interp_membership(velocidad.universe, velocidad['baja'].mf, vel)
    act_vel_media = fuzz.interp_membership(velocidad.universe, velocidad['media'].mf, vel)
    act_vel_alta = fuzz.interp_membership(velocidad.universe, velocidad['alta'].mf, vel)
    act_luz_apagada = fuzz.interp_membership(luz_freno.universe, luz_freno['apagada'].mf, luz)
    act_luz_encendida = fuzz.interp_membership(luz_freno.universe, luz_freno['encendida'].mf, luz)

    r1 = np.fmin(np.fmin(act_dist_corta, act_luz_encendida), act_vel_alta)
    r2 = np.fmin(act_dist_corta, act_vel_alta)
    r3 = np.fmin(act_dist_media, act_luz_encendida)
    r4 = np.fmin(act_dist_larga, act_vel_baja)
    r5 = np.fmin(act_luz_apagada, act_vel_baja)
    r6 = np.fmin(act_dist_media, act_vel_alta)
    r7 = np.fmin(act_dist_corta, act_luz_apagada)

    out1 = np.fmin(r1, frenado['fuerte'].mf)
    out2 = np.fmin(r2, frenado['fuerte'].mf)
    out3 = np.fmin(r3, frenado['moderado'].mf)
    out4 = np.fmin(r4, frenado['nulo'].mf)
    out5 = np.fmin(r5, frenado['nulo'].mf)
    out6 = np.fmin(r6, frenado['moderado'].mf)
    out7 = np.fmin(r7, frenado['moderado'].mf)

    agregado = np.fmax(out1, np.fmax(out2, np.fmax(out3, np.fmax(out4, np.fmax(out5, np.fmax(out6, out7))))))

    if np.sum(agregado) == 0:
        resultado = None
        print("⚠️ Ninguna regla se activó.")
    else:
        resultado = fuzz.defuzz(x_frenado, agregado, metodo)
        print(f"✅ Distancia: {dist} m | Luz: {'Encendida' if luz else 'Apagada'} | Velocidad: {vel} km/h")
        print(f"🛑 Frenado recomendado ({metodo}): {resultado:.2f} %")

    plt.figure(figsize=(10, 6))
    plt.plot(x_frenado, frenado['nulo'].mf, 'b--', label='Frenado nulo')
    plt.plot(x_frenado, frenado['moderado'].mf, 'g--', label='Frenado moderado')
    plt.plot(x_frenado, frenado['fuerte'].mf, 'r--', label='Frenado fuerte')

    plt.fill_between(x_frenado, out1, color='red', alpha=0.2, label='Regla 1')
    plt.fill_between(x_frenado, out2, color='red', alpha=0.2, label='Regla 2')
    plt.fill_between(x_frenado, out3, color='green', alpha=0.2, label='Regla 3')
    plt.fill_between(x_frenado, out4, color='blue', alpha=0.2, label='Regla 4')
    plt.fill_between(x_frenado, out5, color='blue', alpha=0.2, label='Regla 5')
    plt.fill_between(x_frenado, out6, color='green', alpha=0.2, label='Regla 6')
    plt.fill_between(x_frenado, out7, color='green', alpha=0.2, label='Regla 7')

    plt.plot(x_frenado, agregado, 'k-', linewidth=2, label='Agregado')
    if resultado is not None:
        plt.axvline(resultado, color='purple', linestyle='--', label=f'Salida: {resultado:.2f} %')

    plt.title(f'Reglas activadas y salida ({metodo})')
    plt.xlabel('Nivel de Frenado (%)')
    plt.ylabel('Grado de pertenencia')
    plt.legend()
    plt.grid(True)
    plt.show()

# DASHBOARD INTERACTIVO
interact(
    fuzzy_frenado,
    dist=FloatSlider(min=0, max=100, step=1, value=30, description='Distancia (m)'),
    luz=Dropdown(options=[('Apagada', 0), ('Encendida', 1)], value=1, description='Luz de freno'),
    vel=FloatSlider(min=0, max=150, step=1, value=60, description='Velocidad (km/h)'),
    metodo=Dropdown(
        options=[
            ('Centroide', 'centroid'),
            ('Media de los máximos', 'mom'),
            ('Mínimo de los máximos', 'som'),
            ('Máximo de los máximos', 'lom')
        ],
        value='centroid',
        description='Defuzzificación'
    )
);



interactive(children=(FloatSlider(value=30.0, description='Distancia (m)', step=1.0), Dropdown(description='Lu…